Lo primero de todo, cargamos todas las librerías necesarias. 

In [18]:
import pandas as pd
import numpy as np

import sys
import os
import re

import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

In [11]:
import snappy
print(snappy.__version__)

0.7.3


In [12]:
import sys
print(sys.executable)

/Users/angelg.villanueva/Documents/04 DATA-CB-PROGRAM/DS102024_4/.venv/bin/python


In [15]:
# contenido = snappy.decompress(open("data/train.snappy", "rb").read()) if os.path.exists("data/train.snappy") else "Archivo no encontrado"

In [2]:
pd.read_parquet('data/train.snappy')

,descripcion
0,**La observación** 367 describe una seta de *...
1,**La observación** 5123 describe una seta de ...
2,**La observación** 2864 describe una seta de ...
3,**La observación** 4674 describe una seta de ...
4,**La observación** 1210 describe una seta de ...
...,...
4598,**La observación** 1028 describe una seta de ...
4599,**La observación** 5591 describe una seta de ...
4600,**La observación** 2053 describe una seta de ...
4601,**La observación** 4604 describe una seta de ...


In [62]:
pd.set_option('display.max_colwidth', None)
pd.read_parquet('data/train.snappy').iloc[0:3]

,descripcion
0,"**La observación** 367 describe una seta de **clase** edible. Tiene una **forma de sombrero** bell con una **superficie** scaly **y un color** yellow. **Presenta** moretones y tiene un **olor** almond. **Las branquias son de tipo** free con una **separación** close **y un tamaño** broad. **El color de las branquias es** black. **La forma del tallo** es enlarging y la **raíz del tallo** es club. **La superficie del tallo por encima del anillo** es smooth **y por debajo del anillo** es smooth. El **color del tallo por encima del anillo** es white **y por debajo del anillo** es white. **El tipo de velo** es partial con un **color de velo** white. Tiene one **anillo(s)**, **de tipo** pendant **y el color de la impresión de esporas** es brown. Se encuentra en una **población** scattered y su **hábitat es** grasses."
1,"**La observación** 5123 describe una seta de **clase** poisonous. Tiene una **forma de sombrero** convex con una **superficie** smooth **y un color** red. **No presenta** moretones y tiene un **olor** foul. **Las branquias son de tipo** free con una **separación** close **y un tamaño** narrow. **El color de las branquias es** buff. **La forma del tallo** es tapering y la **raíz del tallo** es ?. **La superficie del tallo por encima del anillo** es silky **y por debajo del anillo** es silky. El **color del tallo por encima del anillo** es white **y por debajo del anillo** es pink. **El tipo de velo** es partial con un **color de velo** white. Tiene one **anillo(s)**, **de tipo** evanescent **y el color de la impresión de esporas** es white. Se encuentra en una **población** several y su **hábitat es** paths."
2,"**La observación** 2864 describe una seta de **clase** edible. Tiene una **forma de sombrero** convex con una **superficie** fibrous **y un color** green. **Presenta** moretones y tiene un **olor** null. **Las branquias son de tipo** free con una **separación** close **y un tamaño** broad. **El color de las branquias es** pink. **La forma del tallo** es tapering y la **raíz del tallo** es bulbous. **La superficie del tallo por encima del anillo** es smooth **y por debajo del anillo** es smooth. El **color del tallo por encima del anillo** es pink **y por debajo del anillo** es white. **El tipo de velo** es partial con un **color de velo** white. Tiene one **anillo(s)**, **de tipo** pendant **y el color de la impresión de esporas** es brown. Se encuentra en una **población** several y su **hábitat es** wood."


In [ ]:
pd.read_parquet('data/train.snappy').iloc[0]

descripcion    **La observación** 367  describe una seta de **clase** edible.  Tiene una **forma de sombrero** bell  con una **superficie** scaly  **y un color** yellow.  **Presenta**  moretones y tiene un **olor** almond.  **Las branquias son de tipo** free  con una **separación** close  **y un tamaño** broad.  **El color de las branquias es** black.  **La forma del tallo** es enlarging  y la **raíz del tallo** es club.  **La superficie del tallo por encima del anillo** es smooth  **y por debajo del anillo** es smooth.  El **color del tallo por encima del anillo** es white  **y por debajo del anillo** es white.  **El tipo de velo** es partial  con un **color de velo** white.  Tiene one **anillo(s)**,  **de tipo** pendant  **y el color de la impresión de esporas** es brown.  Se encuentra en una **población** scattered  y su **hábitat es** grasses.  
Name: 0, dtype: object

In [91]:
#hecho con Gemini
import re

fila1 = "**La observación** 367 describe una seta de **clase** edible. Tiene una **forma de sombrero** bell con una **superficie** scaly **y un color** yellow. **Presenta** moretones y tiene un **olor** almond. **Las branquias son de tipo** free con una **separación** close **y un tamaño** broad. **El color de las branquias es** black. **La forma del tallo** es enlarging y la **raíz del tallo** es club. **La superficie del tallo por encima del anillo** es smooth y **por debajo del anillo** es smooth. El **color del tallo por encima del anillo** es white y **por debajo del anillo** es white. **El tipo de velo** es partial con un **color de velo** white. Tiene one **anillo(s)**, **de tipo** pendant y **el color de la impresión de esporas** es brown. Se encuentra en una **población** scattered y su **hábitat es** grasses."

diccionario_fila1 = {
    'ID': re.search(r'\*\*La observación\*\*(.*?) describe', fila1).group(1).strip() if re.search(r'\*\*La observación\*\*(.*?) describe', fila1) else None,
    'clase': re.search(r'\*\*clase\*\*(.*?)\. Tiene', fila1).group(1).strip() if re.search(r'\*\*clase\*\*(.*?)\. Tiene', fila1) else None,
    'forma_sombrero': re.search(r'\*\*forma de sombrero\*\*(.*?) con', fila1).group(1).strip() if re.search(r'\*\*forma de sombrero\*\*(.*?) con', fila1) else None,
    'superficie': re.search(r'\*\*superficie\*\*(.*?) \*\*y', fila1).group(1).strip() if re.search(r'\*\*superficie\*\*(.*?) \*\*y', fila1) else None,
    'color': re.search(r'\*\*y un color\*\*(.*?)\. \*\*Presenta', fila1).group(1).strip() if re.search(r'\*\*y un color\*\*(.*?)\. \*\*Presenta', fila1) else None,
    'presenta': re.search(r'\*\*Presenta\*\*(.*?) y', fila1).group(1).strip() if re.search(r'\*\*Presenta\*\*(.*?) y', fila1) else None,
    'olor': re.search(r'\*\*olor\*\*(.*?)\. \*\*Las', fila1).group(1).strip() if re.search(r'\*\*olor\*\*(.*?)\. \*\*Las', fila1) else None,
    'branquias': re.search(r'\*\*Las branquias son de tipo\*\*(.*?) con', fila1).group(1).strip() if re.search(r'\*\*Las branquias son de tipo\*\*(.*?) con', fila1) else None,
    'separacion_branquias': re.search(r'\*\*separación\*\*(.*?) \*\*y', fila1).group(1).strip() if re.search(r'\*\*separación\*\*(.*?) \*\*y', fila1) else None,
    'tamano_branquias': re.search(r'\*\*y un tamaño\*\*(.*?)\. \*\*El', fila1).group(1).strip() if re.search(r'\*\*y un tamaño\*\*(.*?)\. \*\*El', fila1) else None,
    'color_branquias': re.search(r'\*\*El color de las branquias es\*\*(.*?). \*\*La', fila1).group(1).strip() if re.search(r'\*\*El color de las branquias es\*\*(.*?). \*\*La', fila1) else None,
    'forma_tallo': re.search(r'\*\*La forma del tallo\*\*(?:es) (.*?) y', fila1).group(1).strip() if re.search(r'\*\*La forma del tallo\*\*(?:es) (.*?) y', fila1) else None,
    'raiz_tallo': re.search(r'\*\*raíz del tallo\*\*(.*?) \*\*La', fila1).group(1).strip() if re.search(r'\*\*raíz del tallo\*\*(.*?) \*\*La', fila1) else None,
    'superficie_tallo_arriba': re.search(r'\*\*La superficie del tallo por encima del anillo\*\*(.*?) y', fila1).group(1).strip() if re.search(r'\*\*La superficie del tallo por encima del anillo\*\*(.*?) y', fila1) else None,
    'superficie_tallo_abajo': re.search(r'\*\*por debajo del anillo\*\*(.*?) El', fila1).group(1).strip() if re.search(r'\*\*por debajo del anillo\*\*(.*?) El', fila1) else None,
    'color_tallo_arriba': re.search(r'\*\*El color del tallo por encima del anillo\*\*(.*?) y', fila1).group(1).strip() if re.search(r'\*\*El color del tallo por encima del anillo\*\*(.*?) y', fila1) else None,
    'tipo_velo': re.search(r'\*\*El tipo de velo\*\*(.*?) con', fila1).group(1).strip() if re.search(r'\*\*El tipo de velo\*\*(.*?) con', fila1) else None,
    'color_velo': re.search(r'\*\*color de velo\*\*(.*?). Tiene', fila1).group(1).strip() if re.search(r'\*\*color de velo\*\*(.*?). Tiene', fila1) else None,
    'anillo': re.search(r'\*\*anillo\(s\)\*\*(.*?),', fila1).group(1).strip() if re.search(r'\*\*anillo\(s\)\*\*(.*?),', fila1) else None,
    'tipo_anillo': re.search(r'\*\*de tipo\*\*(.*?) y', fila1).group(1).strip() if re.search(r'\*\*de tipo\*\*(.*?) y', fila1) else None,
    'color_impresion_esporas': re.search(r'\*\*el color de la impresión de esporas\*\*(.*?) Se', fila1).group(1).strip() if re.search(r'\*\*el color de la impresión de esporas\*\*(.*?) Se', fila1) else None,
    'poblacion': re.search(r'\*\*población\*\*(.*?) y', fila1).group(1).strip() if re.search(r'\*\*población\*\*(.*?) y', fila1) else None,
    'habitat': re.search(r'\*\*hábitat es\*\*(.*?).', fila1).group(1).strip() if re.search(r'\*\*hábitat es\*\*(.*?).', fila1) else None
}

print(diccionario_fila1)

{'ID': '367', 'clase': 'edible', 'forma_sombrero': 'bell', 'superficie': 'scaly', 'color': 'yellow', 'presenta': 'moretones', 'olor': 'almond', 'branquias': 'free', 'separacion_branquias': 'close', 'tamano_branquias': 'broad', 'color_branquias': 'black', 'forma_tallo': None, 'raiz_tallo': 'es club.', 'superficie_tallo_arriba': 'es smooth', 'superficie_tallo_abajo': 'es smooth.', 'color_tallo_arriba': None, 'tipo_velo': 'es partial', 'color_velo': 'white', 'anillo': '', 'tipo_anillo': 'pendant', 'color_impresion_esporas': 'es brown.', 'poblacion': 'scattered', 'habitat': ''}
